# 🔬 CARIA-SR Hysteresis Validation
**"Hysteresis in Financial Fragility: Structural Memory and Tail Risk in Low-Volatility Regimes"**

Phases:
- **8**: Quantile Regression + Peak Memory
- **9**: Robustness Heatmap
- **12**: Minsky Hedge Backtest
- **13**: Structural Alpha Grid
- **14**: Bootstrap CI + Sharpe/Sortino

**FMP API**: `79fY9wvC9qtCJHcn6Yelf4ilE9TkRMoq`

In [ ]:
# @title 1. Setup
!pip install -q yfinance pandas numpy scipy scikit-learn statsmodels seaborn matplotlib pyarrow requests

from google.colab import drive
drive.mount('/content/drive')

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import requests
import warnings
from datetime import datetime
import statsmodels.formula.api as smf
from sklearn.covariance import LedoitWolf

warnings.filterwarnings('ignore')
np.random.seed(42)
sns.set_style('whitegrid')

# Directories
WORK_DIR = '/content/drive/MyDrive/CARIA'
os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(f'{WORK_DIR}/figures', exist_ok=True)
os.makedirs(f'{WORK_DIR}/tables', exist_ok=True)

FMP_API_KEY = "79fY9wvC9qtCJHcn6Yelf4ilE9TkRMoq"
START_DATE, END_DATE = "2000-01-01", datetime.now().strftime("%Y-%m-%d")
print(f"✅ Output: {WORK_DIR}")

In [ ]:
# @title 2. Download Data (S&P 500 + Market)

# Get S&P 500 tickers
url = f"https://financialmodelingprep.com/api/v3/sp500_constituent?apikey={FMP_API_KEY}"
resp = requests.get(url)
sp500_tickers = [x['symbol'] for x in resp.json()] if resp.status_code == 200 else []
print(f"Downloading {len(sp500_tickers)} stocks...")

# Download in batches
all_prices = []
for i in range(0, len(sp500_tickers), 50):
    batch = sp500_tickers[i:i+50]
    try:
        data = yf.download(batch, start=START_DATE, end=END_DATE, progress=False, auto_adjust=True)['Close']
        all_prices.append(data)
        print(f"   Batch {i//50 + 1}/{(len(sp500_tickers)-1)//50 + 1}")
    except Exception as e:
        print(f"   Error batch {i//50 + 1}: {e}")

prices = pd.concat(all_prices, axis=1).dropna(axis=1, how='all')
prices.to_csv(f'{WORK_DIR}/sp500_prices.csv')

# Market data (VIX, SPY, TLT, 10Y Treasury)
print("Downloading market data...")
market = yf.download(['^VIX', 'SPY', 'TLT', '^TNX'], start=START_DATE, end=END_DATE, progress=False)

market_df = pd.DataFrame({
    'volatility': market['Close']['^VIX'],
    'price': market['Close']['SPY'],
    'tlt': market['Close']['TLT'],
    'treasury_10y': market['Close']['^TNX']
}).dropna()
market_df.index.name = 'Date'
market_df.to_csv(f'{WORK_DIR}/market_validation_data.csv')

print(f"\n✅ Prices: {prices.shape[1]} stocks, {len(prices)} days")
print(f"✅ Market: {len(market_df)} days")

In [ ]:
# @title 3. Calculate Structural Metrics (AR + Entropy) ⏳ ~15 min

def cov_to_corr(S):
    d = np.sqrt(np.diag(S))
    d = np.where(d == 0, 1e-10, d)
    C = S / np.outer(d, d)
    return np.nan_to_num((C + C.T) / 2)

def eig_metrics(C, k_frac=0.2):
    w = np.sort(np.linalg.eigvalsh(C))[::-1]
    w = np.maximum(w, 1e-10)  # Avoid negative eigenvalues
    k = max(1, int(np.ceil(k_frac * len(w))))
    ar = np.sum(w[:k]) / np.sum(w)
    p = w / np.sum(w)
    ent = -np.sum(p * np.log(p + 1e-10)) / np.log(len(w)) if len(w) > 1 else 0.5
    return float(ar), float(ent)

# Calculate returns
returns = np.log(prices).diff()
good_coverage = returns.notna().mean() >= 0.9
returns = returns.loc[:, good_coverage]
print(f"Using {returns.shape[1]} stocks with >90% coverage")

# Rolling structural metrics
window = 252
step = 5
lw = LedoitWolf()

struct = pd.DataFrame(index=returns.index, columns=['absorption_ratio', 'entropy'], dtype=float)

total_steps = (len(returns) - window) // step
for idx, t in enumerate(range(window, len(returns), step)):
    W = returns.iloc[t-window:t]
    W = W.loc[:, W.notna().mean() >= 0.9]
    if W.shape[1] < 100:
        continue
    W = W.apply(lambda s: s.fillna(s.mean()))
    X = W.values - np.nanmean(W.values, axis=0)
    try:
        C = cov_to_corr(lw.fit(X).covariance_)
    except:
        C = np.corrcoef(X, rowvar=False)
        C = np.nan_to_num((C + C.T) / 2)
    ar, ent = eig_metrics(C)
    struct.iloc[t] = [ar, ent]
    if (idx + 1) % 100 == 0:
        print(f"   {idx + 1}/{total_steps} ({(idx+1)/total_steps*100:.0f}%)")

struct = struct.ffill().bfill()
struct.index.name = 'date'
struct.to_csv(f'{WORK_DIR}/caria_structural_metrics.csv')

print(f"\n✅ Structural metrics saved")
print(f"   AR mean: {struct['absorption_ratio'].mean():.4f}")
print(f"   Entropy mean: {struct['entropy'].mean():.4f}")

In [ ]:
# @title Phase 8: The "Regime & Memory" Test (Fixing the Signal)

print("Loading and merging data for Phase 8...")

# Load data
struct_df = pd.read_csv(f'{WORK_DIR}/caria_structural_metrics.csv', index_col='date', parse_dates=True)
market_df = pd.read_csv(f'{WORK_DIR}/market_validation_data.csv', index_col='Date', parse_dates=True)

# Merge
df = struct_df.join(market_df, how='inner').sort_index()

# Calculate Z-Scores and Memory
window_z = 252
window_memory = 60

rolling_mean = df['absorption_ratio'].rolling(window=window_z).mean()
rolling_std = df['absorption_ratio'].rolling(window=window_z).std()
df['absorp_z'] = (df['absorption_ratio'] - rolling_mean) / rolling_std

# Peak Memory (THE KEY INNOVATION)
df['caria_peak'] = df['absorp_z'].rolling(window=window_memory).max()

df = df.dropna()
print(f"Data prepared. Shape: {df.shape}")

# --- REGIME FILTERING ---
low_vol_df = df[df['volatility'] < 20].copy()
low_vol_df['future_ret_22'] = low_vol_df['price'].pct_change(22).shift(-22)
low_vol_df = low_vol_df.dropna()

print(f"Testing on 'Calm Markets' (VIX < 20). Observations: {len(low_vol_df)}")

# --- QUANTILE REGRESSION ---
print("\nRunning Quantile Regression on 'Hidden Risks'...")

mod_vix = smf.quantreg('future_ret_22 ~ volatility', low_vol_df)
res_vix = mod_vix.fit(q=0.05)

mod_struct = smf.quantreg('future_ret_22 ~ volatility + caria_peak', low_vol_df)
res_struct = mod_struct.fit(q=0.05)

print(res_struct.summary())

print(f"\nBase Model (VIX Only) Pseudo R²:      {res_vix.prsquared:.5f}")
print(f"Structural Model (+Peak) Pseudo R²:   {res_struct.prsquared:.5f}")
imp = ((res_struct.prsquared - res_vix.prsquared)/res_vix.prsquared)*100
print(f"🔥 Improvement in Low-Vol Regime:     {imp:.1f}%")

# --- VISUALIZATION ---
subset = df.loc['2019-01-01':'2020-06-01']

fig, ax1 = plt.subplots(figsize=(12, 6))
ax1.plot(subset.index, subset['volatility'], color='gray', linestyle='--', label='VIX')
ax1.set_ylabel('VIX', color='gray')

ax2 = ax1.twinx()
ax2.plot(subset.index, subset['absorp_z'], color='salmon', alpha=0.5, label='Original Signal (Fades)')
ax2.plot(subset.index, subset['caria_peak'], color='darkred', linewidth=3, label='Peak Signal (Memory)')
ax2.set_ylabel('Structure (Z-Score)', color='darkred')

plt.title('Fixing the "Fall": Using Peak Memory to Sustain the Warning')
fig.legend(loc='upper left', bbox_to_anchor=(0.1, 0.9))
plt.tight_layout()
plt.savefig(f'{WORK_DIR}/figures/Figure_PeakMemory.png', dpi=300)
plt.show()

In [ ]:
# @title Phase 9: Robustness Heatmap (Sensitivity Analysis)

windows = [20, 40, 60, 90, 120]
vix_caps = [15, 18, 20, 22, 25]
results_matrix = np.zeros((len(windows), len(vix_caps)))

print("Running Sensitivity Grid...")

# Pre-compute peak signals for all windows
for w in windows:
    df[f'peak_{w}'] = df['absorp_z'].rolling(window=w).max()

for i, w in enumerate(windows):
    for j, v in enumerate(vix_caps):
        subset = df[df['volatility'] < v].copy()
        subset['ret_future'] = subset['price'].pct_change(22).shift(-22)
        subset = subset.dropna()
        
        if len(subset) > 500:
            try:
                mod_base = smf.quantreg('ret_future ~ volatility', subset)
                res_base = mod_base.fit(q=0.05)
                
                mod_struct = smf.quantreg(f'ret_future ~ volatility + peak_{w}', subset)
                res_struct = mod_struct.fit(q=0.05)
                
                imp = ((res_struct.prsquared - res_base.prsquared)/res_base.prsquared) * 100
                results_matrix[i, j] = imp
            except:
                results_matrix[i, j] = 0
        else:
            results_matrix[i, j] = 0

# Save table
sensitivity_df = pd.DataFrame(results_matrix, index=windows, columns=vix_caps)
sensitivity_df.to_csv(f'{WORK_DIR}/tables/Table_Sensitivity.csv')

# Visualization
plt.figure(figsize=(10, 8))
sns.heatmap(results_matrix, annot=True, fmt=".1f", cmap="RdYlGn", 
            xticklabels=vix_caps, yticklabels=windows)
plt.title("Robustness Check: Improvement in Tail Risk Prediction (%)\n(Y=Memory Window, X=VIX Threshold)")
plt.xlabel("Low-Volatility Threshold (VIX < X)")
plt.ylabel("Memory Window (Days)")
plt.tight_layout()
plt.savefig(f'{WORK_DIR}/figures/Figure_RobustnessHeatmap.png', dpi=300)
plt.show()

In [ ]:
# @title Phase 12: The "Smart Hedge" (SPY ↔ TLT Switching)

print("Running 'Smart Switch' Backtest...")

def get_max_drawdown(equity):
    peak = equity.cummax()
    dd = equity / peak - 1.0
    return float(dd.min())

# Prepare backtest dataframe
backtest_df = df.copy()
backtest_df['daily_ret'] = np.log(backtest_df['price']).diff()
backtest_df['tlt_ret'] = np.log(backtest_df['tlt']).diff()

# Treasury daily return from yield
backtest_df['treasury_daily_ret'] = (backtest_df['treasury_10y'] / 100) / 252

# Unsafe state: Low VIX but high structural stress (Peak Z > 1.5)
peak_z = (backtest_df['caria_peak'] - backtest_df['caria_peak'].rolling(252).mean()) / backtest_df['caria_peak'].rolling(252).std()
backtest_df['unsafe_state'] = (backtest_df['volatility'] < 20) & (peak_z > 1.5)

# Strategy returns
backtest_df['smart_ret'] = np.where(backtest_df['unsafe_state'], 
                                    backtest_df['treasury_daily_ret'], 
                                    backtest_df['daily_ret'])

leverage = 1.5
backtest_df['lev_ret'] = np.where(backtest_df['unsafe_state'], 
                                  backtest_df['treasury_daily_ret'],
                                  backtest_df['daily_ret'] * leverage - (0.05/252 * (leverage-1)))

# Cumulative returns
backtest_df['cum_bnh'] = (1 + backtest_df['daily_ret'].fillna(0)).cumprod()
backtest_df['cum_smart'] = (1 + backtest_df['smart_ret'].fillna(0)).cumprod()
backtest_df['cum_lev'] = (1 + backtest_df['lev_ret'].fillna(0)).cumprod()

# Performance metrics
years = len(backtest_df) / 252

dd_bnh = get_max_drawdown(backtest_df['cum_bnh'])
cagr_bnh = (backtest_df['cum_bnh'].iloc[-1])**(1/years) - 1

dd_smart = get_max_drawdown(backtest_df['cum_smart'])
cagr_smart = (backtest_df['cum_smart'].iloc[-1])**(1/years) - 1

dd_lev = get_max_drawdown(backtest_df['cum_lev'])
cagr_lev = (backtest_df['cum_lev'].iloc[-1])**(1/years) - 1

print(f"\n--- RESULTS ---")
print(f"Benchmark (S&P 500):    DD = {dd_bnh:.1%}, CAGR = {cagr_bnh:.1%}")
print(f"Minsky (Smart/Bond):    DD = {dd_smart:.1%}, CAGR = {cagr_smart:.1%}")
print(f"Minsky (1.5x Levered):  DD = {dd_lev:.1%}, CAGR = {cagr_lev:.1%}")
print(f"\nTime in Hedge: {backtest_df['unsafe_state'].mean()*100:.1f}%")

# Plot
plt.figure(figsize=(12, 6))
plt.plot(backtest_df.index, backtest_df['cum_bnh'], label='S&P 500', color='gray', alpha=0.4)
plt.plot(backtest_df.index, backtest_df['cum_smart'], label=f'Smart Hedge CAGR:{cagr_smart:.1%}', color='blue')
plt.plot(backtest_df.index, backtest_df['cum_lev'], label=f'Levered Minsky CAGR:{cagr_lev:.1%}', color='darkgreen', linewidth=2)
plt.title('Turning Safety into Alpha (Smart Switching & Leverage)', fontsize=14)
plt.yscale('log')
plt.legend()
plt.grid(True, alpha=0.2)
plt.savefig(f'{WORK_DIR}/figures/Figure_MinskyHedge.png', dpi=300)
plt.show()

In [ ]:
# @title Phase 13: Structural Alpha Heatmap (Entropy vs Sync)

entropy_pcts = [0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.40, 0.50]
sync_pcts = [0.50, 0.60, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95]
results_grid = np.zeros((len(sync_pcts), len(entropy_pcts)))

print("Running Grid Search on Structure (Entropy vs Sync)...")

subset = df[df['volatility'] < 25].copy()
subset['ret_future'] = subset['price'].pct_change(22).shift(-22)
subset = subset.dropna()

mod_base = smf.quantreg('ret_future ~ volatility', subset)
res_base = mod_base.fit(q=0.05)
r2_base = res_base.prsquared

for i, s_pct in enumerate(sync_pcts):
    s_thresh = subset['absorption_ratio'].quantile(s_pct)
    
    for j, e_pct in enumerate(entropy_pcts):
        e_thresh = subset['entropy'].quantile(e_pct)
        
        subset['structural_signal'] = ((subset['absorption_ratio'] > s_thresh) & 
                                       (subset['entropy'] < e_thresh)).astype(int)
        
        if subset['structural_signal'].sum() > 10:
            try:
                mod_struct = smf.quantreg('ret_future ~ volatility + structural_signal', subset)
                res_struct = mod_struct.fit(q=0.05)
                imp = ((res_struct.prsquared - r2_base) / r2_base) * 100
                results_grid[i, j] = imp
            except:
                results_grid[i, j] = 0
        else:
            results_grid[i, j] = 0

# Plot
plt.figure(figsize=(12, 9))
x_labels = [f"Bottom {int(p*100)}%" for p in entropy_pcts]
y_labels = [f"Top {int(p*100)}%" for p in sync_pcts]

sns.heatmap(results_grid, annot=True, fmt=".1f", cmap="vlag", center=0,
            xticklabels=x_labels, yticklabels=y_labels,
            cbar_kws={'label': 'Improvement over VIX Baseline (%)'})

plt.title('Relative Improvement of CARIA-SR over Benchmark (%)', fontsize=16, fontweight='bold')
plt.xlabel('Entropy Threshold (Low Diversity)', fontsize=12)
plt.ylabel('Synchronization Threshold (High Rigidity)', fontsize=12)
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig(f'{WORK_DIR}/figures/Figure_StructuralAlpha.png', dpi=300)
plt.show()

In [ ]:
# @title Phase 14: JF Empirical Suite (Sharpe, Sortino, Bootstrap)

def calc_risk_metrics(series, rf=0.04):
    rf_daily = (1 + rf)**(1/252) - 1
    excess_ret = series - rf_daily
    
    ann_ret = np.mean(series) * 252
    ann_vol = np.std(series) * np.sqrt(252)
    sharpe = np.mean(excess_ret) / np.std(series) * np.sqrt(252)
    
    downside = excess_ret[excess_ret < 0]
    downside_std = np.std(downside) * np.sqrt(252)
    sortino = np.mean(excess_ret) / downside_std * np.sqrt(252) if downside_std > 0 else 0
    
    return ann_ret, ann_vol, sharpe, sortino

# Calculate metrics
metrics_bench = calc_risk_metrics(backtest_df['daily_ret'].dropna())
metrics_smart = calc_risk_metrics(backtest_df['smart_ret'].dropna())
metrics_lev = calc_risk_metrics(backtest_df['lev_ret'].dropna())

risk_table = pd.DataFrame({
    'Metric': ['Ann. Return', 'Ann. Volatility', 'Sharpe Ratio', 'Sortino Ratio'],
    'S&P 500': [f"{metrics_bench[0]:.1%}", f"{metrics_bench[1]:.1%}", f"{metrics_bench[2]:.2f}", f"{metrics_bench[3]:.2f}"],
    'Minsky Hedge': [f"{metrics_smart[0]:.1%}", f"{metrics_smart[1]:.1%}", f"{metrics_smart[2]:.2f}", f"{metrics_smart[3]:.2f}"],
    'Minsky 1.5x': [f"{metrics_lev[0]:.1%}", f"{metrics_lev[1]:.1%}", f"{metrics_lev[2]:.2f}", f"{metrics_lev[3]:.2f}"]
})

print("\n--- TABLE: ECONOMIC PERFORMANCE ---")
print(risk_table.to_string(index=False))
risk_table.to_csv(f'{WORK_DIR}/tables/Table_RiskMetrics.csv', index=False)

# --- BOOTSTRAP ---
print("\nRunning Bootstrap (1000 iterations)...")
n_boot = 1000
improvements = []

subset = df[df['volatility'] < 20].copy()
subset['ret_future'] = subset['price'].pct_change(22).shift(-22)
subset = subset.dropna()

for i in range(n_boot):
    sample = subset.sample(n=len(subset), replace=True)
    try:
        mod_base = smf.quantreg('ret_future ~ volatility', sample)
        r2_base = mod_base.fit(q=0.05).prsquared
        
        mod_struct = smf.quantreg('ret_future ~ volatility + caria_peak', sample)
        r2_struct = mod_struct.fit(q=0.05).prsquared
        
        if r2_base > 0:
            imp = (r2_struct - r2_base) / r2_base
            improvements.append(imp)
    except:
        continue

mean_imp = np.mean(improvements)
ci_lower = np.percentile(improvements, 2.5)
ci_upper = np.percentile(improvements, 97.5)

print(f"\n--- BOOTSTRAP RESULTS ---")
print(f"Mean Improvement: {mean_imp:.1%}")
print(f"95% CI: [{ci_lower:.1%}, {ci_upper:.1%}]")
print(f"P(Improvement > 0): {np.mean(np.array(improvements) > 0):.1%}")

In [ ]:
# @title Final Summary

print("="*70)
print("CARIA-SR HYSTERESIS VALIDATION COMPLETE")
print("="*70)

print(f"\n📊 Data:")
print(f"   Universe: {returns.shape[1]} S&P 500 stocks")
print(f"   Period: {df.index.min().date()} to {df.index.max().date()}")
print(f"   Observations: {len(df)}")

print(f"\n🔬 Key Finding (Phase 8):")
print(f"   VIX < 20 Regime Improvement: {imp:.1f}%")

print(f"\n💰 Minsky Hedge (Phase 12):")
print(f"   Max DD Reduction: {dd_bnh - dd_smart:.1%}")
print(f"   Calmar Improvement: {(cagr_smart/abs(dd_smart))/(cagr_bnh/abs(dd_bnh)):.1f}x")

print(f"\n📁 Saved to: {WORK_DIR}")
print(f"   /figures/Figure_*.png")
print(f"   /tables/Table_*.csv")

# Save final dataset
df.to_csv(f'{WORK_DIR}/full_validation_dataset.csv')
print("\n✅ DONE!")